In [3]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from influxdb import *
#import Config as cg
import datetime
from influxdb import DataFrameClient


In [4]:
INFLUX_DB_IP = '10.12.97.178'
INFLUX_DB_PORT = 8086
INFLUX_DB = 'demoDb'
TARGET_MEASUREMENT = 'NN' #

In [5]:
con_obj = InfluxDBClient(host=INFLUX_DB_IP, port=INFLUX_DB_PORT, database=INFLUX_DB)
query = 'select * from ' + TARGET_MEASUREMENT + ' where time > now() - 1d '
df = pd.DataFrame(con_obj.query(query, chunked=True, chunk_size=10000).get_points())
df['time'] = df['time'].astype('datetime64[ns]')

ConnectionError: HTTPConnectionPool(host='10.12.97.178', port=8086): Max retries exceeded with url: /query?q=select+%2A+from+NN+where+time+%3E+now%28%29+-+1d+&db=demoDb&chunked=true&chunk_size=10000 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000026AB173EBA8>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond',))

In [ ]:
df

In [4]:
df.head()

,time,DeviceID,EM_Active Power (kW),EM_Current Avg (A),EM_Current Ph1 (A),EM_Current Ph2 (A),EM_Current Ph3 (A),EM_Energy (kWh),EM_Frequency (Hz),EM_Power Factor,...,EM_Voltage Ph2-Ph3 (V),EM_Voltage Ph3-N (V),Mean_THD,SCS,imbalance_current,imbalance_volt,mean_current,mean_volt,mean_voltage,temp_rise
0,2020-06-03 12:45:33.801754112+00:00,EM23,43638.5,61.1,69.0,56.7,57.6,73069.2,50.0,1.0,...,411.9,239.1,0.0,1,12.929624,0.088990,61.100000,412.033333,237.800000,0.015838
1,2020-06-03 12:45:34.164901120+00:00,EM7,0.0,0.1,0.1,0.1,0.1,24.4,50.0,0.5,...,392.0,226.7,0.0,0,0.000000,0.076472,0.100000,392.300000,226.466667,0.011696
2,2020-06-03 12:45:35.157586944+00:00,EM22,3948.0,6.1,7.0,6.0,5.4,43624.8,50.0,0.9,...,390.8,226.2,0.0,0,14.130435,0.323432,6.133333,391.633333,226.100000,0.209216
3,2020-06-03 12:45:35.199558912+00:00,EM5,315.6,509.0,531.1,485.2,510.6,178095.0,50.0,0.9,...,392.9,227.1,5.4,1,4.348680,0.033921,508.966667,393.066667,226.933333,0.002301
4,2020-06-03 12:45:35.233540096+00:00,EM4,247.4,406.2,403.2,405.6,409.8,140389.0,50.0,0.8,...,394.9,227.9,11.2,1,0.886263,0.202994,406.200000,394.100000,227.533333,0.082413


In [5]:
class Line_Voltage_Mean():

    def __init__(self, a):
        #self.DFDBClient = DataFrameClient(host=cg.INFLUX_DB_IP, port=cg.INFLUX_DB_PORT, database=cg.INFLUX_DB)
        self.a = a
    
    
    def avg_stability(self, df):
        df['diff'] = abs(df.groupby('DeviceID')['mean_current'].diff())
        df = df.fillna(0)
        df['big_spike']=np.where(df["diff"] > (df["diff"].mean() * 1.1), 1, 0)
        e=pd.DataFrame(df[df['big_spike']==1].groupby('DeviceID')['big_spike'].count().astype(np.float64)).reset_index()
        d = pd.DataFrame(df.groupby('DeviceID')['diff'].agg(['mean','min','max'])).reset_index().merge(e,on='DeviceID')
        d=d.rename(columns={"mean": "average_spike", "min": "minimum_spike","max": "maximum_spike","big_spike":"big_spike_count"}) 
        return d
    

    def time_as_index(self, df):
        t = pd.DataFrame(df.groupby(['DeviceID'])['time'].max())
        t.reset_index(inplace=True)
        return t

    def time(self, df):
        df['time'] = pd.to_datetime(df['time'])
        df['Time_max'] = df['time'].dt.time
        a = df.loc[list(df.groupby('DeviceID')['mean_current'].idxmax())][['Time_max', 'DeviceID']]
        b = df.loc[list(df.groupby('DeviceID')['mean_current'].idxmin())][['Time_max', 'DeviceID']]
        a.index = a['DeviceID']
        a = a.drop('DeviceID', axis=1)
        a['Time_min'] = list(b['Time_max'])
        return a
    
    def categorization_time(self, df):
        df['Status'] = 'normal'
        for i in self.a.keys():
            x = df[df['DeviceID'] == i]
            x['Status'] = np.where(x['mean_current'] > self.a[i], 'high', x['Status'])
            df[df['DeviceID'] == i] = x
        return df


    def calculate_count(self, df):
        df = self.categorization_time(df)
        c = pd.DataFrame(df.groupby(['DeviceID', 'Status']).size()).reset_index()
        c = pd.pivot_table(index='DeviceID', columns='Status', values=0, data=c, aggfunc=np.sum).astype(np.float64)
        c = c.rename(columns={"normal": "normal_count","high": "high_count"})
        c.reset_index(inplace=True)
        return c

    
    def duration(self, df):
        x = pd.DataFrame(df.groupby([(df.Status != df.Status.shift()).cumsum()])['time'].apply(
            lambda x: (x.iloc[-1] - x.iloc[0]).total_seconds() / 60))
        x['Status'] = df.loc[df.Status.shift(-1) != df.Status]['Status'].values
        x.reset_index(drop=True, inplace=True) 
        return x
    
    def calculate_time(self,df):
        df = self.categorization_time(df)
        y = df.groupby(['DeviceID']).apply(self.duration).reset_index()
        y = pd.pivot_table(index='DeviceID', columns='Status', values='time', data=y, aggfunc=np.sum).reset_index()
        y = y.rename(columns={"normal": "normal_duration","high": "high_duration"})
        return y

    def summary_data(self,df):
        x = df.groupby('DeviceID')['mean_current'].describe()
        x.columns = ['Total_Count', 'Average', 'SD', 'Minimum', '25th_percentile', 'Median', '75th_percentile',
                     'Maximum']
        x['IQR']= x['75th_percentile']-x['25th_percentile']
        return x

    def zero_duration(self,df):
        t = df.index.to_series().diff()!=1
        t = df.groupby(t.cumsum())['time'].agg(lambda x: abs((x.iloc[-1] - x.iloc[0]).total_seconds() / 60)).sum()
        return t
    
    def scs_count(self,df):
        s = ((df.SCS != df.SCS.shift(axis=0)).sum(axis=0)-1)/2
        return s
    
    def output(self,df):
        q=self.time_as_index(df)
        s = pd.DataFrame(df.groupby('DeviceID').apply(self.scs_count))
        s=s.rename(columns={0: 'scs_count'})
        q = q.merge(s, on = "DeviceID", how = "outer")  
        df0 = df[df['mean_current'] == 0]
        df = df[df['mean_current'] > 0]
        x = self.summary_data(df) 
        x = q.merge(x, on = "DeviceID", how = "outer")
        y = self.calculate_time(df)
        x = x.merge(y, on = "DeviceID", how = "outer")
        z=self.calculate_count(df)
        x = x.merge(z, on = "DeviceID", how = "outer")
        w=self.avg_stability(df)
        x = x.merge(w, on = "DeviceID", how = "outer")
        v=self.time(df)
        x = x.merge(v, on = "DeviceID", how = "outer")
        if(len(df0) != 0):
            o = pd.DataFrame(df0.groupby(['DeviceID']).size(), columns = ['count_0']).reset_index()
            x = x.merge(o, on='DeviceID', how='outer')
            t = pd.DataFrame(df0.groupby(['DeviceID']).apply(self.zero_duration),columns=['Duration_0']).reset_index()
            x = x.merge(t,on='DeviceID',how='outer')
        x = x.fillna(0)
        x.set_index('time', inplace=True)       
        return x


In [6]:
if __name__ == '__main__':
    a = {'EM1': 1635, 'EM2': 1020, 'EM3': 1020, 'EM4': 820, 'EM5': 820, 'EM6': 83, 'EM7': 156, 'EM8': 160, 'EM9': 160,
     'EM10': 160, 'EM22': 23.5, 'EM23': 33}
    cat = Line_Voltage_Mean(a)
    t = cat.output(df)
    print(t)

                                    DeviceID  scs_count  Total_Count  \
time                                                                   
2020-06-04 12:04:13.443040+00:00         EM1        0.0      24104.0   
2020-06-04 11:56:43.157250816+00:00     EM10        3.0      17023.0   
2020-06-04 11:56:57.268750080+00:00      EM2        0.0          0.0   
2020-06-04 11:56:59.693107968+00:00     EM22        0.0       9566.0   
2020-06-04 12:45:29.862417920+00:00     EM23        1.5       4543.0   
2020-06-04 11:56:57.806428928+00:00      EM4        1.0      27496.0   
2020-06-04 12:04:04.414937088+00:00      EM5        0.0      26664.0   
2020-06-04 12:45:31.812997120+00:00      EM6        0.0      21473.0   
2020-06-04 12:45:30.923310080+00:00      EM7        0.0       7249.0   
2020-06-04 11:55:24.490305792+00:00      EM9        4.0       6760.0   

                                        Average          SD     Minimum  \
time                                                        